# 2026年银价预测数据分析

## 数据概述
本数据集包含2026年银价的预测数据，包括预测价格以及置信区间的上下限。

### 数据字段
- **Date**: 日期
- **Predicted_Price**: 预测价格
- **Lower_Bound**: 预测下限
- **Upper_Bound**: 预测上限

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# 设置显示中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 读取数据
df = pd.read_csv('silver_price_forecast_2026.csv')

# 转换日期格式
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Month_Name'] = df['Date'].dt.month_name()
df['Weekday'] = df['Date'].dt.weekday
df['Weekday_Name'] = df['Date'].dt.day_name()

# 显示前几行数据
print("数据集形状:", df.shape)
print("\n前5行数据:")
df.head()

In [ ]:
# 数据基本信息
print("数据类型:")
print(df.dtypes)
print("\n缺失值统计:")
print(df.isnull().sum())
print("\n数据描述统计:")
print(df.describe())

## 数据分析结果

### 1. 价格走势分析

In [ ]:
# 价格统计
price_stats = {
    '平均预测价': df['Predicted_Price'].mean(),
    '最高预测价': df['Predicted_Price'].max(),
    '最低预测价': df['Predicted_Price'].min(),
    '价格标准差': df['Predicted_Price'].std(),
    '价格波动幅度': df['Predicted_Price'].max() - df['Predicted_Price'].min()
}

print("价格统计:")
for key, value in price_stats.items():
    print(f"{key}: ${value:.2f}")

In [ ]:
# 价格走势图
plt.figure(figsize=(15, 6))
plt.plot(df['Date'], df['Predicted_Price'], label='预测价格', linewidth=2, color='steelblue')
plt.fill_between(df['Date'], df['Lower_Bound'], df['Upper_Bound'], 
                 alpha=0.3, label='置信区间', color='steelblue')
plt.title('2026年银价预测走势')
plt.xlabel('日期')
plt.ylabel('价格 ($/oz)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 2. 价格波动分析

In [ ]:
# 计算价格变化率
df['Price_Change'] = df['Predicted_Price'].diff()
df['Price_Change_Pct'] = df['Predicted_Price'].pct_change() * 100

# 波动统计
volatility_stats = {
    '平均日变化': df['Price_Change'].mean(),
    '最大日涨幅': df['Price_Change'].max(),
    '最大日跌幅': df['Price_Change'].min(),
    '平均日变化率': df['Price_Change_Pct'].mean(),
    '价格波动率': df['Predicted_Price'].std()
}

print("价格波动统计:")
for key, value in volatility_stats.items():
    if '率' in key:
        print(f"{key}: {value:.2f}%")
    else:
        print(f"{key}: ${value:.2f}")

In [ ]:
# 价格变化可视化
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# 日变化
axes[0].bar(df['Date'], df['Price_Change'], width=1)
axes[0].axhline(y=0, color='red', linestyle='--')
axes[0].set_title('每日价格变化')
axes[0].set_ylabel('变化 ($/oz)')
axes[0].grid(True, alpha=0.3)
axes[0].tick_params(axis='x', rotation=45)

# 日变化率
colors = ['green' if x >= 0 else 'red' for x in df['Price_Change_Pct'].dropna()]
axes[1].bar(df['Date'][1:], df['Price_Change_Pct'].dropna(), width=1, color=colors)
axes[1].axhline(y=0, color='black', linestyle='--')
axes[1].set_title('每日价格变化率')
axes[1].set_ylabel('变化率 (%)')
axes[1].grid(True, alpha=0.3)
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

### 3. 置信区间分析

In [ ]:
# 置信区间宽度
df['CI_Width'] = df['Upper_Bound'] - df['Lower_Bound']
df['CI_Width_Pct'] = (df['CI_Width'] / df['Predicted_Price']) * 100

# 置信区间统计
ci_stats = {
    '平均置信区间宽度': df['CI_Width'].mean(),
    '最大置信区间宽度': df['CI_Width'].max(),
最小置信区间宽度': df['CI_Width'].min(),
    '平均置信区间占比': df['CI_Width_Pct'].mean()
}

print("置信区间统计:")
for key, value in ci_stats.items():
    if '占比' in key:
        print(f"{key}: {value:.2f}%")
    else:
        print(f"{key}: ${value:.2f}")

In [ ]:
# 置信区间宽度变化
plt.figure(figsize=(15, 6))
plt.plot(df['Date'], df['CI_Width'], label='置信区间宽度', linewidth=2, color='coral')
plt.axhline(y=df['CI_Width'].mean(), color='red', linestyle='--', 
            label=f'平均值: ${df["CI_Width"].mean():.2f}')
plt.title('置信区间宽度变化')
plt.xlabel('日期')
plt.ylabel('宽度 ($/oz)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 4. 月度趋势分析

In [ ]:
# 按月统计
monthly_stats = df.groupby('Month').agg({
    'Predicted_Price': ['mean', 'min', 'max', 'std'],
    'Price_Change': 'sum',
    'CI_Width': 'mean'
})
monthly_stats.columns = ['平均价', '最低价', '最高价', '价格标准差', '累计变化', '平均置信宽度']

print("月度价格统计:")
print(monthly_stats)

In [ ]:
# 月度趋势可视化
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 平均价格
axes[0, 0].bar(monthly_stats.index, monthly_stats['平均价'], color='steelblue')
axes[0, 0].set_title('月度平均价格')
axes[0, 0].set_xlabel('月份')
axes[0, 0].set_ylabel('价格 ($/oz)')
axes[0, 0].grid(True, alpha=0.3, axis='y')

# 价格范围
axes[0, 1].plot(monthly_stats.index, monthly_stats['最低价'], label='最低价', marker='o')
axes[0, 1].plot(monthly_stats.index, monthly_stats['最高价'], label='最高价', marker='o')
axes[0, 1].fill_between(monthly_stats.index, monthly_stats['最低价'], monthly_stats['最高价'], alpha=0.3)
axes[0, 1].set_title('月度价格范围')
axes[0, 1].set_xlabel('月份')
axes[0, 1].set_ylabel('价格 ($/oz)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 累计变化
colors_m = ['green' if x >= 0 else 'red' for x in monthly_stats['累计变化']]
axes[1, 0].bar(monthly_stats.index, monthly_stats['累计变化'], color=colors_m)
axes[1, 0].axhline(y=0, color='black', linestyle='--')
axes[1, 0].set_title('月度累计价格变化')
axes[1, 0].set_xlabel('月份')
axes[1, 0].set_ylabel('累计变化 ($/oz)')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# 价格波动
axes[1, 1].plot(monthly_stats.index, monthly_stats['价格标准差'], marker='s', color='purple')
axes[1, 1].set_title('月度价格波动')
axes[1, 1].set_xlabel('月份')
axes[1, 1].set_ylabel('标准差')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 5. 周期性分析

In [ ]:
# 按星期几统计
weekday_stats = df.groupby('Weekday_Name').agg({
    'Predicted_Price': ['mean', 'std'],
    'Price_Change': 'mean'
})
weekday_stats.columns = ['平均价', '标准差', '平均日变化']

# 按星期顺序排序
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
weekday_stats = weekday_stats.loc[weekday_order]

print("按星期几统计:")
print(weekday_stats)

In [ ]:
# 星期几价格对比
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 平均价格
weekday_stats['平均价'].plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('按星期几的平均价格')
axes[0].set_ylabel('价格 ($/oz)')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3, axis='y')

# 平均日变化
colors_wd = ['green' if x >= 0 else 'red' for x in weekday_stats['平均日变化']]
weekday_stats['平均日变化'].plot(kind='bar', ax=axes[1], color=colors_wd)
axes[1].axhline(y=0, color='black', linestyle='--')
axes[1].set_title('按星期几的平均日变化')
axes[1].set_ylabel('日变化 ($/oz)')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

### 6. 风险评估

In [ ]:
# 风险指标计算
risk_metrics = {
    '价格波动率': df['Predicted_Price'].std() / df['Predicted_Price'].mean() * 100,
    '最大下跌幅度': (df['Predicted_Price'].max() - df['Predicted_Price'].min()) / df['Predicted_Price'].max() * 100,
    '平均风险暴露': df['CI_Width'].mean() / df['Predicted_Price'].mean() * 100,
    '单日最大跌幅': df['Price_Change'].min() / df['Predicted_Price'].mean() * 100,
    '单日最大涨幅': df['Price_Change'].max() / df['Predicted_Price'].mean() * 100
}

print("风险评估指标:")
for key, value in risk_metrics.items():
    print(f"{key}: {value:.2f}%")

## 数据场景

### 1. 投资决策
- **应用场景**: 基于价格预测制定投资策略
- **价值**: 识别买入和卖出时机，优化投资收益

### 2. 风险管理
- **应用场景**: 利用置信区间评估投资风险
- **价值**: 制定风险控制策略，设置止损点

### 3. 套期保值
- **应用场景**: 企业预测未来价格，制定套保策略
- **价值**: 规避价格波动风险，锁定成本

### 4. 资产配置
- **应用场景**: 根据价格预测调整贵金属资产配置
- **价值**: 优化投资组合，分散风险

### 5. 生产计划
- **应用场景**: 银相关企业基于价格预测制定生产计划
- **价值**: 优化生产节奏，提高利润

### 6. 市场研究
- **应用场景**: 研究贵金属价格周期性特征
- **价值**: 揭示市场规律，指导交易策略

### 7. 预算规划
- **应用场景**: 企业基于价格预测制定采购预算
- **价值**: 合理规划资金，控制成本

## 主要发现

1. **价格趋势**: 2026年银价呈现一定的波动趋势
2. **波动特征**: 价格存在明显的波动性，需要风险管理
3. **置信区间**: 预测存在不确定性，置信区间宽度反映预测精度
4. **季节性**: 可能存在月度或季节性模式
5. **风险水平**: 价格波动率和风险暴露需要重点关注

## 数据质量
- 数据完整性: 无缺失值
- 数据准确性: 基于模型预测，存在不确定性
- 数据时效性: 预测数据，需要结合实时市场情况

## 建议

1. **风险管理**: 根据置信区间设置合理的止损和止盈点
2. **分散投资**: 不要将所有资金投入单一贵金属
3. **定期复盘**: 跟踪实际价格与预测的偏差，调整策略
4. **关注基本面**: 结合经济指标和市场事件综合判断
5. **长期视角**: 避免短期波动影响长期投资决策
6. **专业咨询**: 重要决策建议咨询专业投资顾问

## 注意事项

1. 预测模型存在局限性，实际价格可能偏离预测
2. 金融市场受多种因素影响，不确定性较高
3. 投资有风险，决策需谨慎
4. 建议结合多种分析方法进行判断
5. 本数据仅供参考，不构成投资建议